In [2]:
import tensorflow.keras as keras
import numpy as np
import importlib

import kaggle_functions as kaggle
import model9_deepervgg2
importlib.reload(kaggle); 
importlib.reload(model9_deepervgg2);
from model9_deepervgg2 import Model

model_number = 'model9'

### Create datasets

In [2]:
train_dataset, valid_dataset, valid_labels = kaggle.load_train_as_dataset()

batch_size = 32
train_dataset_augmented, epoch_length = kaggle.augment_dataset(train_dataset, batch_size)

### Train new model

In [ ]:
Model().build(None).summary()

In [ ]:
# Fit model
# See {model}.py for specific training instructions, like epochs or valid_patience
model_name = ''

model, history = kaggle.train_model(
    Model().build(None), train_dataset_augmented, valid_dataset, 
    epochs=250, valid_patience=25, epoch_length=epoch_length)

In [ ]:
# Save model
model.save(f'models/{model_number}/{model_name}')

# Plot model statistics during training
kaggle.plot_model_history(history, [['accuracy', 'val_accuracy'], ['loss', 'val_loss']])

### Fine-tune

In [ ]:
# Fine-tune model
fine_model, history = kaggle.fine_tune_model(
    model,
    train_dataset.batch(32), valid_dataset, 
    epochs=5, learning_rate=0.00001)

In [ ]:
# Save fine-tuned model
fine_model_name = ''
fine_model.save(f'models/{model_number}/{fine_model_name}')

### Evaluate

In [ ]:
model_name = ''
model = keras.models.load_model(f'models/{model_number}/{model_name}')
test_pred = np.argmax(model.predict(valid_dataset.batch(128)), axis=1)

kaggle.print_accuracy(valid_labels, test_pred)
kaggle.plot_confusion_matrix(valid_labels, test_pred)

### Generate test predictions

In [ ]:
model_name = ''
kaggle.generate_test_pred_filepath(f'models/{model_number}/{model_name}')

### Hypertune models

In [ ]:
tuner = kaggle.hypertune_model(
    Model(), train_dataset_augmented, valid_dataset, 
    model_number, 'hypertuner2021-11-30', epochs=200, trials=5)

In [ ]:
tuner = kaggle.load_hypertuner(Model(), model_number, '')